In [ ]:
!pip install --upgrade diffusers transformers streamlit -q


In [ ]:

from pathlib import Path
import torch
from diffusers import StableDiffusionPipeline
import streamlit as st
from PIL import Image


class TIG:
    device = "cuda" if torch.cuda.is_available() else "cpu"
    seed = 42
    generator = torch.Generator(device).manual_seed(seed)
    img_gen_steps = 35
    img_gen_model_id = "CompVis/stable-diffusion-v1-4"
    img_gen_size = (398, 398)
    img_gen_guidance_scale = 9


img_gen_model = StableDiffusionPipeline.from_pretrained(
    TIG.img_gen_model_id, torch_dtype=torch.float16, revision="fp16",img_gen_guidance_scale=9
)
img_gen_model = img_gen_model.to(TIG.device)

def generate_image(prompt, model):
    image = model(
        prompt, num_inference_steps=TIG.img_gen_steps,
        generator=TIG.generator,
        guidance_scale=TIG.img_gen_guidance_scale
    ).images[0]

    image = image.resize(TIG.img_gen_size)
    image_path = "generated_image.png"
    image.save(image_path)
    return image_path

def main():
    st.title("Text-to-Image Generation with Stable Diffusion")

    text_prompt = st.text_input("Enter your text prompt here:")

    if st.button("Generate Image"):
        if text_prompt:
            image_path = generate_image(text_prompt, img_gen_model)
            image = Image.open(image_path)
            st.image(image, caption='Generated Image', use_column_width=True)
        else:
            st.error("Please enter a text prompt.")

if __name__ == '__main__':
    main()


In [ ]:
!pip install pyngrok -q
from pyngrok import ngrok


ngrok.kill()

public_url = ngrok.connect(port='8501')
print(f"Streamlit app running at: {public_url}")

!streamlit run app.py --server.port 8501
